### Predicting formation enthalpies for solid solutions of Lanthanides Orthophosphates


In [1]:
import os
import sys
import inspect

print(sys.version)

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
grandparentdir = os.path.dirname(parentdir)

sys.path.insert(0, grandparentdir) 

3.9.6 (default, Nov 10 2023, 13:38:27) 
[Clang 15.0.0 (clang-1500.1.0.2.5)]


In [2]:
import read_data
import featureSpan
import lasso
import Utils
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import scipy.optimize
import pandas as pd
import seaborn as sns
import scipy.stats as ss
from sklearn.linear_model import Lasso
from itertools import combinations, product
import itertools
import math
import pandas as pd

## Xenotime 

#### Load data from files

In [3]:
list1 = read_data.readData("../../data/DATA_HE_xenotime.dat")
print("Shape of List1 is {}".format(list1.shape))
print(list1)

Shape of List1 is (525, 4)
[[5.70000000e+01 5.80000000e+01 7.50000000e-01 2.31598225e-01]
 [5.70000000e+01 5.80000000e+01 6.25000000e-01 2.78958493e-01]
 [5.70000000e+01 5.80000000e+01 5.00000000e-01 2.97583757e-01]
 ...
 [7.00000000e+01 7.10000000e+01 5.00000000e-01 7.77590152e-02]
 [7.00000000e+01 7.10000000e+01 3.75000000e-01 7.09266168e-02]
 [7.00000000e+01 7.10000000e+01 2.50000000e-01 5.91211365e-02]]


In [4]:
list2 = read_data.readCSVData("../../data/Data_Ln-xenotime.csv", material="xenotime", Volume=False)
print("Shape of List2 is {}".format(list2.shape))
print((list2))

Shape of List2 is (15, 8)
[[ 57.      138.90547  19.1773   91.7      11.        1.1      49.95
    1.16   ]
 [ 58.      140.116    20.198    99.73     12.        1.12     36.758
    1.143  ]
 [ 59.      140.90765  21.624   112.27     12.3       1.13     38.98
    1.126  ]
 [ 60.      144.242    22.1     120.19     12.95      1.14     40.41
    1.109  ]
 [ 61.      145.       22.3     127.26     13.6       1.13     41.1
    1.093  ]
 [ 62.      150.36     23.4     137.16     14.25      1.17     41.4
    1.079  ]
 [ 63.      151.964    24.92    143.83     14.9       1.2      42.7
    1.066  ]
 [ 64.      157.25     20.63    149.42     15.9       1.2      44.
    1.053  ]
 [ 65.      158.92535  21.91    156.33     16.2       1.1      39.79
    1.04   ]
 [ 66.      162.5      22.8     163.       16.85      1.22     41.47
    1.027  ]
 [ 67.      164.93032  22.84    169.12     17.5       1.23     42.5
    1.015  ]
 [ 68.      167.259    22.74    177.09     18.15      1.24     42.7
    1.004

#### Generated Elemental Training/Testing sets from loaded data

In [5]:
def generateFeatures_full2(HEList, featureList):
    X = []
    Y = []
    
    for i in range(len(HEList)):
        Y.append(HEList[i][3])
        
        x = HEList[i][2]
        X1 = featureList[int(HEList[i][0]-57)]
        X2 = featureList[int(HEList[i][1]-57)]

        #X.append(np.concatenate((np.array([x, 1/x, x * x, 1 / (x * x), 1-x, 1/(1-x), (1-x) * (1-x), 1/( (1-x) * (1-x))]), np.array([abs(X1-X2)/2.0, 2.0 / (abs(X1-X2)), (X1 + X2) / 2.0, 2.0 / (X1 + X2)]).flatten())))
        mfeatures = np.array([x, 1/x, x * x, 1 / (x * x), 1-x, 1/(1-x), (1-x) * (1-x), 1/( (1-x) * (1-x))])
        
        f1 = []
        f2 = []
        f3 = []
        f4 = []
        
        for j in range(len(X1)-3):
            f1.append([abs(X1[j]-X2[j])/2.0])
            f2.append([2.0/abs(X1[j]-X2[j])])
            f3.append([(X1[j] + X2[j]) / 2.0])
            f4.append([2.0 / (X1[j] + X2[j])])
            
        #rint(f3)
        #print(np.asarray(f3).flatten())
    
        #print(np.concatenate((mfeatures, np.asarray(f1).flatten(), np.asarray(f3).flatten())))
              
        f5 = []
        f6 = []
        f7 = []
        
        for j in range(5,7):
            f5.append([abs(X1[j]-X2[j])/2.0])
            f6.append([(X1[j] + X2[j]) / 2.0])
            f7.append([2.0 / (X1[j] + X2[j])])            
            
        f8 = []
        f9 = []
        f10 = []
        f11 = []
        f12 = []
        f13 = []
        f14 = []
        f15 = []
        f16 = []
        f17 = []
        f18 = []
        f19 = []
        
        for j in range(7,len(X1)):
            f8.append([abs(X1[j]-X2[j])/2.0])
            f9.append([2.0/abs(X1[j]-X2[j])])
            f10.append([(X1[j] + X2[j]) / 2.0])
            f11.append([2.0 / (X1[j] + X2[j])])
            f12.append([pow(abs(X1[j]-X2[j])/2.0,2)])
            f13.append([pow(2.0/abs(X1[j]-X2[j]),2)])
            f14.append([pow((X1[j] + X2[j]) / 2.0,2)])
            f15.append([pow(2.0 / (X1[j] + X2[j]),2)])
            f16.append([pow(abs(X1[j]-X2[j])/2.0,3)])
            f17.append([pow(2.0/abs(X1[j]-X2[j]),3)])
            f18.append([pow((X1[j] + X2[j]) / 2.0,3)])
            f19.append([pow(2.0 / (X1[j] + X2[j]),3)])
            
        X.append(np.concatenate((mfeatures, np.asarray(f1).flatten(),  np.asarray(f2).flatten(),  np.asarray(f3).flatten(), np.asarray(f4).flatten(), np.asarray(f5).flatten(), np.asarray(f6).flatten(),np.asarray(f7).flatten(),np.asarray(f8).flatten(),np.asarray(f9).flatten(),np.asarray(f10).flatten(),np.asarray(f11).flatten(),np.asarray(f12).flatten(),np.asarray(f13).flatten(),np.asarray(f14).flatten(),np.asarray(f15).flatten(),np.asarray(f16).flatten(),np.asarray(f17).flatten(),np.asarray(f18).flatten(),np.asarray(f19).flatten()     )))
            
    return X, Y

In [6]:
listX, listY = generateFeatures_full2(list1, list2)
X = np.asarray(listX)
Y = np.asarray(listY)
print(X.shape)
print(Y.shape)

elemental_list = ["m", "1/m", "m^2", "(1/m)^2", "(1-m)", "1/(1-m)", "(1-m)^2", "(1/(1-m))^2", "[AD:Z]", "[AD:M]", "[AD:IP2]", "[AD:Young]", "[AD:Charge]",  "(1/[AD:Z])", "(1/[AD:M])", "(1/[AD:IP2])", "(1/[AD:Young])", "(1/[AD:Charge])",  "[AM:Z]", "[AM:M]", "[AM:IP2]", "[AM:Young]", "[AM:Charge]",  "(1/[AM:Z])", "(1/[AM:M])", "(1/[AM:IP2])", "(1/[AM:Young])", "(1/[AM:Charge])", "[AD:electronegativity]", "[AD:IP3]", "[AM:electronegativity]", "[AM:IP3]","(1/[AM:electronegativity])", "(1/[AM:IP3])" ,    "[AD:R]", "(1/[AD:R])","[AM:R]","(1/[AM:R])",   "([AD:R])^2", "(1/[AD:R])^2" , "([AM:R])^2","(1/[AM:R])^2",    "([AD:R])^3", "(1/[AD:R])^3", "([AM:R])^3", "(1/[AM:R])^3"         ]
print(len(elemental_list))
print(elemental_list)
m, n = X.shape

elemental_features = []

for i in elemental_list:
    elemental_features.append("("+i+")")
        
print("The elemental features are: \n {}".format(elemental_features))

(525, 46)
(525,)
46
['m', '1/m', 'm^2', '(1/m)^2', '(1-m)', '1/(1-m)', '(1-m)^2', '(1/(1-m))^2', '[AD:Z]', '[AD:M]', '[AD:IP2]', '[AD:Young]', '[AD:Charge]', '(1/[AD:Z])', '(1/[AD:M])', '(1/[AD:IP2])', '(1/[AD:Young])', '(1/[AD:Charge])', '[AM:Z]', '[AM:M]', '[AM:IP2]', '[AM:Young]', '[AM:Charge]', '(1/[AM:Z])', '(1/[AM:M])', '(1/[AM:IP2])', '(1/[AM:Young])', '(1/[AM:Charge])', '[AD:electronegativity]', '[AD:IP3]', '[AM:electronegativity]', '[AM:IP3]', '(1/[AM:electronegativity])', '(1/[AM:IP3])', '[AD:R]', '(1/[AD:R])', '[AM:R]', '(1/[AM:R])', '([AD:R])^2', '(1/[AD:R])^2', '([AM:R])^2', '(1/[AM:R])^2', '([AD:R])^3', '(1/[AD:R])^3', '([AM:R])^3', '(1/[AM:R])^3']
The elemental features are: 
 ['(m)', '(1/m)', '(m^2)', '((1/m)^2)', '((1-m))', '(1/(1-m))', '((1-m)^2)', '((1/(1-m))^2)', '([AD:Z])', '([AD:M])', '([AD:IP2])', '([AD:Young])', '([AD:Charge])', '((1/[AD:Z]))', '((1/[AD:M]))', '((1/[AD:IP2]))', '((1/[AD:Young]))', '((1/[AD:Charge]))', '([AM:Z])', '([AM:M])', '([AM:IP2])', '([AM:

In [7]:
dfX = pd.DataFrame(data=X, columns=elemental_features)

new_features=[]
new_columns = []

for i in range(n):
    for j in range(i):
        new_features.append(elemental_features[i]+"*"+elemental_features[j])
        new_columns.append(dfX[elemental_features[i]].values * dfX[elemental_features[j]].values)


for i in range(n):
    for j in range(i):
        for k in range(j):
            new_features.append(elemental_features[i]+"*"+elemental_features[j]+"*"+elemental_features[k])
            new_columns.append(dfX[elemental_features[i]] * dfX[elemental_features[j]] * dfX[elemental_features[k]])
            
            
new_columns = np.asarray(new_columns)
dfX = pd.concat(
    [
        dfX,
        pd.DataFrame(
            new_columns.T, 
            index=dfX.index, 
            columns=new_features
        )
    ], axis=1
)

dfX.head()

dfX.std() == 0
dfX=dfX.loc[:, dfX.std() > 0]
dfX.head()

,(m),(1/m),(m^2),((1/m)^2),((1-m)),(1/(1-m)),((1-m)^2),((1/(1-m))^2),([AD:Z]),([AD:M]),...,((1/[AM:R])^3)*(([AM:R])^3)*([AD:R]),((1/[AM:R])^3)*(([AM:R])^3)*((1/[AD:R])),((1/[AM:R])^3)*(([AM:R])^3)*([AM:R]),((1/[AM:R])^3)*(([AM:R])^3)*((1/[AM:R])),((1/[AM:R])^3)*(([AM:R])^3)*(([AD:R])^2),((1/[AM:R])^3)*(([AM:R])^3)*((1/[AD:R])^2),((1/[AM:R])^3)*(([AM:R])^3)*(([AM:R])^2),((1/[AM:R])^3)*(([AM:R])^3)*((1/[AM:R])^2),((1/[AM:R])^3)*(([AM:R])^3)*(([AD:R])^3),((1/[AM:R])^3)*(([AM:R])^3)*((1/[AD:R])^3)
0,0.750,1.333333,0.562500,1.777778,0.250,4.000000,0.062500,16.000000,0.5,0.605265,...,0.0085,117.647059,1.1515,0.868432,0.000072,13840.83045,1.325952,0.754175,6.141250e-07,1.628333e+06
1,0.625,1.600000,0.390625,2.560000,0.375,2.666667,0.140625,7.111111,0.5,0.605265,...,0.0085,117.647059,1.1515,0.868432,0.000072,13840.83045,1.325952,0.754175,6.141250e-07,1.628333e+06
2,0.500,2.000000,0.250000,4.000000,0.500,2.000000,0.250000,4.000000,0.5,0.605265,...,0.0085,117.647059,1.1515,0.868432,0.000072,13840.83045,1.325952,0.754175,6.141250e-07,1.628333e+06
3,0.375,2.666667,0.140625,7.111111,0.625,1.600000,0.390625,2.560000,0.5,0.605265,...,0.0085,117.647059,1.1515,0.868432,0.000072,13840.83045,1.325952,0.754175,6.141250e-07,1.628333e+06
4,0.250,4.000000,0.062500,16.000000,0.750,1.333333,0.562500,1.777778,0.5,0.605265,...,0.0085,117.647059,1.1515,0.868432,0.000072,13840.83045,1.325952,0.754175,6.141250e-07,1.628333e+06


In [8]:
print(len(dfX.columns.values))
np.array(dfX.columns.values)

16255


array(['(m)', '(1/m)', '(m^2)', ...,
       '((1/[AM:R])^3)*(([AM:R])^3)*((1/[AM:R])^2)',
       '((1/[AM:R])^3)*(([AM:R])^3)*(([AD:R])^3)',
       '((1/[AM:R])^3)*(([AM:R])^3)*((1/[AD:R])^3)'], dtype=object)

In [9]:
def LassoFit(lmb, X, Y, max_iter=100000, standardization = True):
    
    scaler = StandardScaler()
    scaler.fit(X)
    X_standardized = scaler.transform(X)
    lasso =  Lasso(alpha=lmb, max_iter=max_iter)
    lasso.fit(X_standardized, Y.copy())
    coef =  lasso.coef_
    selected_indices = coef.nonzero()[0]
    selected_features = np.array(dfX.columns.values)[selected_indices]
    Y_predict = lasso.predict(X_standardized)
    MAE, MSE, ME = Utils.compute_error(Y.copy(), Y_predict)
        
    return coef, selected_indices, selected_features, MAE, MSE, ME

LassoFit(0.01, dfX, Y)

(array([-0., -0., -0., ..., -0.,  0.,  0.]),
 array([ 1257,  3452,  3752,  3986,  4794,  4830,  5117,  5647,  5906,
         5917,  5987,  7282,  9528,  9861,  9921, 10002, 11253, 12572,
        12614, 12650, 12657, 12859, 12869, 12894, 13024, 14806]),
 array(['([AD:Young])*((1-m)^2)*(m^2)',
        '((1/[AM:IP2]))*([AD:Charge])*([AD:Young])',
        '((1/[AM:Young]))*([AD:Charge])*([AD:Young])',
        '((1/[AM:Young]))*((1/[AM:IP2]))*([AD:Young])',
        '([AD:IP3])*([AD:Young])*([AD:IP2])',
        '([AD:IP3])*((1/[AD:M]))*([AD:IP2])',
        '([AD:IP3])*([AD:electronegativity])*([AD:IP2])',
        '([AM:IP3])*([AD:Charge])*([AD:Young])',
        '([AM:IP3])*((1/[AM:Young]))*([AD:Young])',
        '([AM:IP3])*((1/[AM:Young]))*([AM:Charge])',
        '([AM:IP3])*([AD:IP3])*([AD:Young])',
        '([AD:R])*([AM:Young])*((1/[AD:Z]))',
        '(([AD:R])^2)*((1-m)^2)*(m^2)',
        '(([AD:R])^2)*((1/[AM:Young]))*((1/[AM:IP2]))',
        '(([AD:R])^2)*([AD:IP3])*((1-m))',
        

In [10]:
#### Define a function which fits Lasso to have no more nonzero coefficients than a given threshold 
def LassoSelect(X, Y, min, max, step, threshold, standardization = True):
    
    scaler = StandardScaler()
    scaler.fit(X)
    X_standardized = scaler.transform(X)
        
    found = False
    for lmbda in np.arange (min, max, step):
        coef, selected_indices, selected_features, MAE, MSE, ME = LassoFit(lmbda, X.copy(), Y.copy())
        if len(selected_indices) <= threshold:
            found = True
            break
    
    if found:
        print("FOUND with threshold: {}".format(threshold))
        print("Lambda: {}, nnz: {}, MAE: {}, MSE: {}, MAPE: {}".format(lmbda, len(selected_indices), MAE, MSE, ME))
            
    else:
        print("NOT FOUND with threshold: {}".format(threshold))
        print("Closest are: ")
        print("Lambda: {}, nnz: {}, MAE: {}, MSE: {}, MAPE: {}".format(lmbda, len(selected_indices), MAE, MSE, ME))
     
    X_reduced = X[selected_features]
    
    return X_reduced


X_reduced = LassoSelect(dfX, Y, 0.001, 0.101, 0.005, 30)
X_reduced.head()

FOUND with threshold: 30
Lambda: 0.011, nnz: 26, MAE: 0.07093753463722813, MSE: 0.010646780176452049, MAPE: 0.1081168564068135


,([AD:Young])*((1-m)^2)*(m^2),((1/[AM:IP2]))*([AD:Charge])*([AD:Young]),((1/[AM:Young]))*([AD:Charge])*([AD:Young]),((1/[AM:Young]))*((1/[AM:IP2]))*([AD:Young]),([AD:IP3])*([AD:Young])*([AD:IP2]),([AD:IP3])*((1/[AD:M]))*([AD:IP2]),([AD:IP3])*([AD:electronegativity])*([AD:IP2]),([AM:IP3])*([AD:Charge])*([AD:Young]),([AM:IP3])*((1/[AM:Young]))*([AD:Young]),([AM:IP3])*((1/[AM:Young]))*([AM:Charge]),...,(([AM:R])^2)*((1/[AM:M]))*([AM:Charge]),(([AD:R])^3)*((1-m)^2)*(m^2),(([AD:R])^3)*([AD:Young])*((1-m)),(([AD:R])^3)*((1/[AD:M]))*((1-m)),(([AD:R])^3)*((1/[AD:M]))*([AD:Young]),(([AD:R])^3)*((1/[AM:IP2]))*((1-m)),(([AD:R])^3)*((1/[AM:IP2]))*((1/[AD:M])),(([AD:R])^3)*((1/[AM:Young]))*((1/[AD:M])),(([AD:R])^3)*([AM:IP3])*((1-m)),(([AM:R])^3)*([AM:IP3])*([AM:Charge])
0,0.141152,0.101967,0.020974,0.002131,13.515568,5.561644,0.033663,87.033155,1.81859,5.208912,...,0.109299,2.159033e-08,6.164280e-07,2.536596e-07,0.000004,7.798353e-09,5.153679e-08,1.060062e-08,0.000007,761.235162
1,0.220551,0.101967,0.020974,0.002131,13.515568,5.561644,0.033663,87.033155,1.81859,5.208912,...,0.109299,3.373489e-08,9.246420e-07,3.804893e-07,0.000004,1.169753e-08,5.153679e-08,1.060062e-08,0.000010,761.235162
2,0.250938,0.101967,0.020974,0.002131,13.515568,5.561644,0.033663,87.033155,1.81859,5.208912,...,0.109299,3.838281e-08,1.232856e-06,5.073191e-07,0.000004,1.559671e-08,5.153679e-08,1.060062e-08,0.000013,761.235162
3,0.220551,0.101967,0.020974,0.002131,13.515568,5.561644,0.033663,87.033155,1.81859,5.208912,...,0.109299,3.373489e-08,1.541070e-06,6.341489e-07,0.000004,1.949588e-08,5.153679e-08,1.060062e-08,0.000017,761.235162
4,0.141152,0.101967,0.020974,0.002131,13.515568,5.561644,0.033663,87.033155,1.81859,5.208912,...,0.109299,2.159033e-08,1.849284e-06,7.609787e-07,0.000004,2.339506e-08,5.153679e-08,1.060062e-08,0.000020,761.235162


In [11]:
def LassoL0(X, Y, nnz):    
    nr, nc = X.shape
    X = np.column_stack((X, np.ones(nr)))
    se_min = np.inner(Y, Y)
    coef_min, permu_min = None, None
    for permu in combinations(range(nc), nnz):
        X_ls = X[:, permu + (-1,)]
        coef, se, __1, __2 = np.linalg.lstsq(X_ls, Y, rcond=-1)
        try:
            if se[0] < se_min: 
                se_min = se[0]
                coef_min, permu_min = coef, permu
        except:
            pass
        
    return coef_min, permu_min

In [12]:
def LassoL0Fit(X_reduced, Y, nnz, log=True):
    
    scaler = StandardScaler()
    scaler.fit(X_reduced)
    X_std = scaler.transform(X_reduced)
    
    nr, nc = X_reduced.shape
   
    coefficients, selected_indices = LassoL0(X_std, Y, nnz)
   
    coefficients = np.array(coefficients)
    selected_indices = np.array(selected_indices)
    feature_reduced = np.array(X_reduced.columns.values)
    feature_list_selected = feature_reduced[selected_indices]
    
    X_selected = X_reduced[feature_list_selected]

    mean_selected = X_selected.mean()
    std_selected = X_selected.std()

    
    if log:
        print("Lasso: selected coefficients are: {}".format(coefficients))
        print("Lasso: selected features are: {}".format(feature_list_selected))
        
    #-mean/std
    mean_std = []
    for i in range(len(selected_indices)):
        mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
 
    sum_mean_std = sum(mean_std)

    for i in range(len(selected_indices)):
        coefficients[i] = coefficients[i] / std_selected[i]

    
    coefficients[len(selected_indices)] -= sum_mean_std
    
    function = str(coefficients[0])+" * "+feature_list_selected[0]
    
    for i in range(1, len(selected_indices)):
        if coefficients[i] >= 0:
            function += " + " + str(coefficients[i])+" * "+feature_list_selected[i]
        else:
            function += " - " + str(abs(coefficients[i]))+" * "+feature_list_selected[i]

    
    if coefficients[len(selected_indices)] >= 0:
        function += " + " + str(coefficients[len(selected_indices)])
    else:
        function += " - " + str(abs(coefficients[len(selected_indices)]))
    
    if log:
        print("Constructed function is: {}".format(function))

    X_selected = np.column_stack((X_selected, np.ones(X_selected.shape[0])))
    Y_predict = X_selected[:,0] * coefficients[0]

    for i in range(1,len(selected_indices)+1):
        Y_predict = Y_predict + X_selected[:,i] * coefficients[i]
    
    if log:
        Utils.print_error(Y.copy(),Y_predict,"Lasso L0: {} coef".format(nnz))
    
    return Y_predict, coefficients, selected_indices

In [13]:
LassoL0Fit(X_reduced, Y.copy(), 1);

Lasso: selected coefficients are: [3.64653817 3.1736172 ]
Lasso: selected features are: ['((1/[AM:Young]))*([AD:Charge])*([AD:Young])']
Constructed function is: 9.82820752232656 * ((1/[AM:Young]))*([AD:Charge])*([AD:Young]) - 0.03179579887527595
Lasso L0: 1 coef
Mean absolute error: 0.3575702781985611
Mean squared error: 0.3691348953868166
Mean absolute percentage error: 0.16787773452585164


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_21493/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_21493/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [14]:
LassoL0Fit(X_reduced, Y.copy(), 2);

Lasso: selected coefficients are: [1.82150906 1.9192058  3.1736172 ]
Lasso: selected features are: ['(([AD:R])^2)*((1-m)^2)*(m^2)' '(([AD:R])^2)*([AM:IP3])*((1/[AM:Young]))']
Constructed function is: 18347.919603277292 * (([AD:R])^2)*((1-m)^2)*(m^2) + 3136.838600238836 * (([AD:R])^2)*([AM:IP3])*((1/[AM:Young])) - 0.010956691135454832
Lasso L0: 2 coef
Mean absolute error: 0.1547098875895086
Mean squared error: 0.06034466423159909
Mean absolute percentage error: 0.1044182422167305


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_21493/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_21493/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [15]:
LassoL0Fit(X_reduced, Y.copy(), 3);

Lasso: selected coefficients are: [1.79287458 1.60640694 0.37739746 3.1736172 ]
Lasso: selected features are: ['(([AD:R])^2)*((1-m)^2)*(m^2)' '(([AD:R])^2)*([AM:IP3])*((1/[AM:Young]))'
 '(([AD:R])^3)*((1/[AM:IP2]))*((1-m))']
Constructed function is: 18059.486747469135 * (([AD:R])^2)*((1-m)^2)*(m^2) + 2625.5857033602074 * (([AD:R])^2)*([AM:IP3])*((1/[AM:Young])) + 93534.58787268901 * (([AD:R])^3)*((1/[AM:IP2]))*((1-m)) + 0.05773510741176713
Lasso L0: 3 coef
Mean absolute error: 0.13215650107826118
Mean squared error: 0.0335012377085014
Mean absolute percentage error: 0.19029724815026985


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_21493/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_21493/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [16]:
LassoL0Fit(X_reduced, Y.copy(), 4);

Lasso: selected coefficients are: [0.8490876  1.70668543 0.90627483 0.34267195 3.1736172 ]
Lasso: selected features are: ['((1/[AM:IP2]))*([AD:Charge])*([AD:Young])'
 '(([AD:R])^2)*((1-m)^2)*(m^2)'
 '(([AD:R])^3)*((1/[AM:Young]))*((1/[AD:M]))'
 '(([AD:R])^3)*([AM:IP3])*((1-m))']
Constructed function is: 0.3421794401715304 * ((1/[AM:IP2]))*([AD:Charge])*([AD:Young]) + 17191.31007646736 * (([AD:R])^2)*((1-m)^2)*(m^2) + 12680655.290462801 * (([AD:R])^3)*((1/[AM:Young]))*((1/[AD:M])) + 89.94281682409309 * (([AD:R])^3)*([AM:IP3])*((1-m)) + 0.006647530034561644
Lasso L0: 4 coef
Mean absolute error: 0.08716906859277339
Mean squared error: 0.016239769984368754
Mean absolute percentage error: 0.09159493065063049


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_21493/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_21493/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [17]:
LassoL0Fit(X_reduced, Y.copy(), 5);

Lasso: selected coefficients are: [0.85388325 0.13691162 1.72811369 0.74789401 0.35006443 3.1736172 ]
Lasso: selected features are: ['((1/[AM:IP2]))*([AD:Charge])*([AD:Young])'
 '([AM:IP3])*([AD:IP3])*([AD:Young])' '(([AD:R])^2)*((1-m)^2)*(m^2)'
 '(([AD:R])^3)*((1/[AM:Young]))*((1/[AD:M]))'
 '(([AD:R])^3)*([AM:IP3])*((1-m))']
Constructed function is: 0.34411207017256756 * ((1/[AM:IP2]))*([AD:Charge])*([AD:Young]) + 6.561304921491548e-05 * ([AM:IP3])*([AD:IP3])*([AD:Young]) + 17407.155335792464 * (([AD:R])^2)*((1-m)^2)*(m^2) + 10464580.754606044 * (([AD:R])^3)*((1/[AM:Young]))*((1/[AD:M])) + 91.8831564104997 * (([AD:R])^3)*([AM:IP3])*((1-m)) + 0.005408340193160388
Lasso L0: 5 coef
Mean absolute error: 0.08138228997104635
Mean squared error: 0.013945302698035872
Mean absolute percentage error: 0.09182408354369362


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_21493/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_21493/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]
